In [2]:
import pandas as pd
import os

In [12]:
import numpy as np

In [3]:

for file in os.listdir('./raw_news/'):
    print(file)


bs.csv
et_news.csv
mint.csv
NewsData-2023-02-19.csv
NewsData-2023-02-20.csv
NewsData-2023-02-21.csv
NewsData-2023-02-22.csv
NewsData-2023-02-23.csv
NewsData-2023-02-24.csv
NewsData-2023-02-25.csv
NewsData-2023-02-26.csv


In [7]:
# importing libraries
import pandas as pd
import glob
import os

# merging the files
joined_files = os.path.join("./raw_news/", "*.csv")
print(joined_files)
# A list of all joined files is returned
joined_list = glob.glob(joined_files)
print(joined_files)
# Finally, the files are joined
df = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
print(df)


./raw_news/*.csv
./raw_news/*.csv
       Unnamed: 0                                           Headline  \
0               0  Sebi to control unsolicited fin market advise ...   
1               1  Sebi pitches change in rules for REITs, InvITs...   
2               2  Sri Lanka's body approves renewable energy pro...   
3               3  Sebi likely to scrap small town-linked incenti...   
4               4  Indices post biggest weekly decline since June...   
...           ...                                                ...   
31500          24  LIC's investment in Adani Group stocks now sta...   
31501          25  PolicyBazaar hits over 5-month high; surges 30...   
31502          26  Jindal Stainless (Hisar) fixes record date for...   
31503          27  Mahindra Lifespace tumbles 8% after MD & CEO t...   
31504          28  Siemens scaling new heights on strong performa...   

                            Date  
0      February 24, 2023, Friday  
1      February 24, 2023, Frida

In [8]:
df.head()

,Unnamed: 0,Headline,Date
0,0,Sebi to control unsolicited fin market advise ...,"February 24, 2023, Friday"
1,1,"Sebi pitches change in rules for REITs, InvITs...","February 24, 2023, Friday"
2,2,Sri Lanka's body approves renewable energy pro...,"February 24, 2023, Friday"
3,3,Sebi likely to scrap small town-linked incenti...,"February 24, 2023, Friday"
4,4,Indices post biggest weekly decline since June...,"February 24, 2023, Friday"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31505 entries, 0 to 31504
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Headline  31505 non-null  object
 1   Date      31505 non-null  object
dtypes: object(2)
memory usage: 492.4+ KB


In [10]:
df.drop(columns='Unnamed: 0', inplace=True)

In [13]:
df['Sentiment'] = np.random.randint(0,1)

In [15]:
df.head()

,Headline,Date,Sentiment
0,Sebi to control unsolicited fin market advise ...,"February 24, 2023, Friday",0
1,"Sebi pitches change in rules for REITs, InvITs...","February 24, 2023, Friday",0
2,Sri Lanka's body approves renewable energy pro...,"February 24, 2023, Friday",0
3,Sebi likely to scrap small town-linked incenti...,"February 24, 2023, Friday",0
4,Indices post biggest weekly decline since June...,"February 24, 2023, Friday",0


In [17]:
df.to_csv('./news/stock.csv')